In [1]:
import numpy as np
import pandas as pd

from bokeh.plotting import figure, output_file, show, curdoc
from bokeh.models import ColumnDataSource, Grid, Line, Plot, CrosshairTool
from bokeh.layouts import gridplot

In [2]:
filename = '../data/training_data.csv'
training_data = pd.read_csv(filename)

In [3]:
def make_interactive_plots(wellname):
    output_file(wellname+'_interactive_well.html')
    
    line_colors = ['red', 'blue', 'seagreen', 'orangered', 'black']
    x_labels = ['GAMMA RAY (API)', 'Log10(RESISTIVITY) (ohm-m)', 'dPHI (%)', 'POROSITY_ND (%)', 'PHOTOELECTRIC EFFECT']
    
    # get info for specific training well:
    df = training_data[training_data['Well Name'] == wellname]
    
    datasource = ColumnDataSource({'x1':np.array(df.GR),
                                   'x2':np.array(df.ILD_log10),
                                   'x3':np.array(df.DeltaPHI),
                                   'x4':np.array(df.PHIND),
                                   'x5':np.array(df.PE),
                                   'y' :np.array(df.Depth)})
    
    # set a template for subsequent linked plots:
    fig0 = figure(x_axis_location="above")
    fig0.y_range.flipped = True
    fig0.line('x1', 'y', line_width = 2, line_color = 'red', source=datasource)
    
    # Info shown on mouse-over using HoverTool:
    TOOLTIPS = [("index", "$index"),
                ("(x,y)", "($x, $y)")]
    
    # all y_range's set to pre-defined figure for linked behavior:
    plots = [figure(x_axis_location="above", y_range=fig0.y_range, 
                    x_axis_label = x_labels[i], y_axis_label = 'DEPTH (ft)', tooltips=TOOLTIPS) for i in range(5)]
    [plot.line('x'+str(i+1), 'y', color=line_colors[i], line_width=2, source=datasource) for i,plot in enumerate(plots)]
    
    # add a cross-hair tool for more specific viewing:
    for i,plot in enumerate(plots):
        plot.add_tools(CrosshairTool(dimensions="both"))
    
    show(gridplot(children=[plot for plot in plots], ncols=5, 
                  plot_height=1000, plot_width=250, sizing_mode = 'stretch_height', toolbar_location='right'))

In [4]:
# generate interactive plots per training well:
well_names = ['CHURCHMAN BIBLE', 'CROSS H CATTLE', 'LUKE G U', 'NEWBY', 'NOLAN', 'SHRIMPLIN']

for well in well_names:
    make_interactive_plots(well)